In [8]:
import pandas as pd
import geopandas as gpd

In [9]:
import folium

In [14]:
df = pd.read_csv("./Data/Calenviroscreen4.csv", dtype={
    'Census Tract': str,
    'California County': str,
    'Approximate Location': str
})
df.head()

FileNotFoundError: [Errno 2] No such file or directory: './Data/Calenviroscreen4.csv'

In [ ]:
df['Census Tract'] = df['Census Tract'].str.zfill(11)

In [ ]:
df.info()

In [ ]:
columns_to_keep = ['Census Tract',
                   'Total Population',
                   'California County',
                   'Approximate Location',
                   'CES 4.0 Score',
                   'CES 4.0 Percentile',
                   'CES 4.0 Percentile Range',
                   'PM2.5 Pctl',
                   'Groundwater Threats Pctl',
                   'Asthma Pctl',
                   'Low Birth Weight Pctl',
                   'Unemployment Pctl',
                   'Housing Burden Pctl',
                   'Pollution Burden Pctl',
                   'Drinking Water Pctl',
                   'Tox. Release Pctl',
                   'Cleanup Sites Pctl',
                   'Poverty Pctl',
                   'Housing Burden Pctl',
                   'Cardiovascular Disease Pctl']
df2 = df[columns_to_keep]
df2.info

In [ ]:
df2LA = df2[df2['California County'] == 'Los Angeles']

In [ ]:
df2LA.rename(columns = {'Census Tract':'FIPS'}, inplace = True) 

In [ ]:
df2LA.info()

In [ ]:
tracts = gpd.read_file('2020_Census_Tracts.geojson')
tracts.head()

In [ ]:
tracts['FIPS'] ='06' + '037' + tracts['CT20']

In [ ]:
CEStracts=tracts.merge(df2LA,on='FIPS')

In [ ]:
CEStracts.crs

In [ ]:
dfoil = pd.read_csv('Oil_Wells.csv')

In [ ]:
df_act = dfoil.drop(dfoil[dfoil['WellStatus'] != 'A'].index)

In [ ]:
gdf_act = gpd.GeoDataFrame(df_act, 
                       crs='epsg:4326',
                       geometry=gpd.points_from_xy(df_act.Longitude, df_act.Latitude))

In [ ]:
# Perform a spatial join based on geographic coordinates
gdfoilCES = gpd.sjoin(gdf_act, CEStracts, how='right', op='within')
gdfoilCES.head()

In [ ]:
gdfoilCES.info()

In [ ]:
oilwellsbytract = gdfoilCES.FIPS.value_counts().rename_axis('FIPS').reset_index(name='oilwell_count')

In [ ]:
oilwellsbytract.info()

In [ ]:
# join the summary table back to the gdf
gdfoilCES=gdfoilCES.merge(oilwellsbytract,on='FIPS')

In [ ]:
gdfoilCES.info()

In [ ]:
gdfoilCES.head()

In [ ]:
# Check for duplicated column names
duplicated_columns = gdfoilCES.columns[gdfoilCES.columns.duplicated()]
if len(duplicated_columns) > 0:
    print("Duplicated columns found:", duplicated_columns)
    # Drop duplicated columns or rename them to make them unique
    # For example:
    # gdfoilCES = gdfoilCES.drop(columns=duplicated_columns)
    # Or rename duplicated columns:
    # gdfoilCES = gdfoilCES.rename(columns={'duplicate_column': 'new_name'})


In [ ]:
# add empty folium map
m = folium.Map(location=[34.2,-118.2], 
               zoom_start = 9,
               tiles='CartoDB positron', 
               attribution='CartoDB')

# plot chorpleth over the base map
folium.Choropleth(
                  geo_data=gdfoilCES, # geo data
                  data=gdfoilCES, # data          
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS','CES 4.0 Score'], # [key, value]
                  fill_color='Oranges',
                  line_weight=0.1, 
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='Extremely Low Income Households (2022)').add_to(m)

# create feature group
f2=folium.FeatureGroup(name='Oil Wells').add_to(m)

# add the wells to the feature group
for index, row in gdf_act.iterrows():
    folium.Circle(
        radius=1,
        color="black",
        location=[row.Latitude,row.Longitude],
        overlay=False).add_to(f2)

folium.LayerControl(position='topright',collapsed=True, autoZIndex=True).add_to(m)

m